In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [2]:
teams = 8
placements = 5

r = (500, 300, 200, 100, 50) # points awarded for tour 3

teamnames = ['beastcoast', 'EG', 'Thunder Awaken', 'Infinity', 'Infamous', 'Hokori', 'Mad Kings', 'Balrogs']
c = (700, 970, 143, 80, 32, 60, 0, 0) # current DPC points (after Berlin Major)

In [3]:
model = gp.Model('Example 3: DPC SA 2023 Tour 3 Division I') # initialize model

# define variables
x = model.addMVar((teams, placements), vtype=GRB.BINARY, name="regional league decision variables") 
d = model.addMVar((teams), vtype=GRB.INTEGER, name="DPC points") 
q = model.addMVar((teams), vtype=GRB.BINARY, name="is qualified for Bali") # bool if team is qualified for the last major

# formulate constraints
for t in range(teams):
    model.addConstr(x[t, :].sum() <= 1)
    model.addConstr(q[t] == x[t,0] + x[t,1], "Bali_qual_constr")
    model.addConstr(d[t] == (gp.quicksum((x[t,p]*r[p]) for p in range(placements))) + c[t] )
                    
for p in range(placements):                 
    model.addConstr(x[:, p].sum() == 1)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-21


In [4]:
model.setObjective(d[1], GRB.MAXIMIZE) # define objective function, here we maximize EGs points as an example
model.optimize() # solve

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 29 rows, 56 columns and 152 nonzeros
Model fingerprint: 0xa351db1f
Variable types: 0 continuous, 56 integer (48 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 16 rows and 16 columns
Presolve time: 0.00s
Presolved: 13 rows, 40 columns, 80 nonzeros
Variable types: 0 continuous, 40 integer (40 binary)

Root relaxation: objective 1.470000e+03, 9 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1470.0000000 1470.00000  0.00%     -    0s

Explored 1 nodes (9 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 4 (

In [5]:
for t in range(teams): # results
    print('%s, %s points' % (teamnames[t], d.X[t]))

beastcoast, 800.0 points
EG, 1470.0 points
Thunder Awaken, 143.0 points
Infinity, 80.0 points
Infamous, 332.0 points
Hokori, 60.0 points
Mad Kings, 50.0 points
Balrogs, 200.0 points


In [6]:
x.X.astype(int) # evaluate matrix of decision variables

array([[0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0]])